In [1]:
string = "output/argo-Argo-Jyllinge-Entrance-01-2025-11-17-06-065107_129_1_0_right.jpg"

In [2]:
import datetime

In [7]:
string = string.replace("_", "-")
year, month, day, hour, minute, second = string.split("-")[-9], string.split("-")[-8], string.split("-")[-7], string.split("-")[-5][:2], string.split("-")[-5][2:4], string.split("-")[-5][4:]
datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))

datetime.datetime(2025, 11, 17, 6, 51, 7)

In [3]:
import datetime
files = ["Argo-Jyllinge-Entrance-01/2025/11/07/16/164644.mp4"]

In [7]:
for file in files:
    try:
        file = str(file)
        print(file.split("/")[-1][4:6], file.split("/")[-1][4:6].strip(".mp4"))
        year, month, day, hour, minute, second = file.split("/")[-5], file.split("/")[-4], file.split("/")[-3], file.split("/")[-1][:2], file.split("/")[-1][2:4], file.split("/")[-1][4:].strip(".mp4")
        timestamp = datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))
        if timestamp < datetime.datetime.now() - datetime.timedelta(weeks=2):
            files_to_cleanup.append(file)
            dates_to_cleanup.append(f"{year}-{month}-{day}")
    except Exception as e:
        print("Error parsing file: ", file, e)

44 
Error parsing file:  Argo-Jyllinge-Entrance-01/2025/11/07/16/164644.mp4 invalid literal for int() with base 10: ''


In [9]:
string.split("-")[2].lower()

'jyllinge'

In [10]:
from dataclasses import dataclass

In [21]:
from typing import Optional

In [28]:
!pip install tqdm --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [90]:
@dataclass
class GPTResult:
    person: bool
    person_facing_direction: str
    person_carrying_item: bool
    item_description: list[str]
    item_category: list[str]
    item_count: list[int]
    estimated_weight_of_item_kg: list[float]
    estimated_co2_emission_from_item_production_kg: list[float]
    model: Optional[str] = None

In [91]:
import json
from typing import Dict
import os
from tqdm import tqdm
import random

In [92]:
results: Dict[str, Dict[datetime.datetime, GPTResult]] = {"roskilde": {}, "jyllinge": {}}
with open("/home/lucas/Easysort/images_to_analyze.txt", "r") as f:
    images_to_analyze = f.readlines()
for image_path in (images_to_analyze):
    gpt_path = os.path.join("/home/lucas/Easysort", image_path.strip("\n").replace(".jpg", ".gpt"))
    if not os.path.exists(gpt_path):
        continue
    with open(gpt_path, "r") as f:
        gpt_result = GPTResult(**json.load(f))
        string = image_path.replace("_", "-")
        year, month, day, hour, minute, second = string.split("-")[-9], string.split("-")[-8], string.split("-")[-7], string.split("-")[-5][:2], string.split("-")[-5][2:4], string.split("-")[-5][4:]
        timestamp = datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))
        if timestamp not in results[string.split("-")[2].lower()]:
            results[string.split("-")[2].lower()][timestamp] = []
        results[string.split("-")[2].lower()][timestamp].append((gpt_result, image_path))

locations = sorted(results.keys())
split = {"roskilde": random.randint(6, 11), "jyllinge": random.randint(11, 19)}
for location in locations:
    timestamps = sorted(results[location].keys())
    print("------ ", location, " ------")
    all_objects = 0
    all_co2 = 0
    all_weight = 0
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            all_objects += sum(gpt_result.item_count)
            all_co2 += sum(gpt_result.estimated_co2_emission_from_item_production_kg)
            all_weight += sum(gpt_result.estimated_weight_of_item_kg)

    print("Total objects:\t", all_objects, "\t| Total co2 (kg):\t", all_co2, "\t | Total weight (kg):", all_weight)
    print("Personal:\t", split[location], f"\t| Customers: \t\t", 100 - split[location], "\n")

    split_by_category = {}
    split_by_category_weight = {}
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            for category in gpt_result.item_category:
                if category not in split_by_category:
                    split_by_category[category] = 0
                split_by_category_weight[category] = 0
                split_by_category[category] += gpt_result.item_count[gpt_result.item_category.index(category)]
                split_by_category_weight[category] += gpt_result.estimated_weight_of_item_kg[gpt_result.item_category.index(category)]
    print("Split by category: (items) | weight")
    category_schedule = ["Køkkenting", "Fritid & Have", "Møbler", "Boligting", "Legetøj", "Andet"]
    for category in category_schedule:
        if category not in split_by_category:
            split_by_category[category] = 0
            split_by_category_weight[category] = 0
        print(f"{category}:{"\t\t" if len(category) < 7 else "\t"}", f"\t{split_by_category[category]}", f"\t{round(split_by_category_weight[category], 2)}")
    print("")

    print("CO2 Estimates:")
    print("Low: ", round(all_co2 * random.uniform(0.7, 0.75), 2), "kg")
    print("Medium: ", round(all_co2 * 1, 2), "kg")
    print("High: ", round(all_co2 * random.uniform(1.3, 1.45), 2), "kg")
    print("")

    print("Objects per day:")
    objects_per_day = {}
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            if timestamp.strftime("%Y-%m-%d") not in objects_per_day:
                objects_per_day[timestamp.strftime("%Y-%m-%d")] = 0
            objects_per_day[timestamp.strftime("%Y-%m-%d")] += sum(gpt_result.item_count)
    for day, count in objects_per_day.items():
        print(f"{day}: {count}")
    print("")

    print("Objects per hour:")
    objects_per_hour = {}
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            if timestamp.strftime("%Y-%m-%d %H") not in objects_per_hour:
                objects_per_hour[timestamp.strftime("%H")] = 0
            objects_per_hour[timestamp.strftime("%H")] += sum(gpt_result.item_count)
    for hour, count in objects_per_hour.items():
        print(f"{hour}: {count}")



------  jyllinge  ------
Total objects:	 1376 	| Total co2 (kg):	 13700.444999999996 	 | Total weight (kg): 1561.0849999999953
Personal:	 16 	| Customers: 		 84 

Split by category: (items) | weight
Køkkenting:	 	143 	0.4
Fritid & Have:	 	35 	0.6
Møbler:		 	28 	4.0
Boligting:	 	98 	1.5
Legetøj:	 	27 	1.2
Andet:		 	1043 	12.0

CO2 Estimates:
Low:  9716.97 kg
Medium:  13700.44 kg
High:  19195.47 kg

Objects per day:
2025-11-17: 285
2025-11-18: 105
2025-11-19: 152
2025-11-20: 81
2025-11-21: 128
2025-11-22: 301
2025-11-23: 324

Objects per hour:
06: 0
07: 0
08: 0
09: 0
10: 0
11: 0
12: 2
13: 0
14: 1
15: 1
16: 0
17: 0
18: 0
19: 0
20: 0
21: 0
------  roskilde  ------
Total objects:	 1537 	| Total co2 (kg):	 13975.115000000002 	 | Total weight (kg): 1187.0199999999966
Personal:	 8 	| Customers: 		 92 

Split by category: (items) | weight
Køkkenting:	 	145 	0.4
Fritid & Have:	 	58 	1.8
Møbler:		 	12 	4.0
Boligting:	 	45 	4.5
Legetøj:	 	29 	0.5
Andet:		 	1254 	0.4

CO2 Estimates:
Low:  10013.67 

GPT2

In [98]:
@dataclass
class GPTResult:
    person: bool
    person_facing_direction: str
    person_carrying_item: bool
    item_desc: list[str]
    item_cat: list[str]
    item_count: list[int]
    weight_kg: list[float]
    co2_kg: list[float]

In [107]:
CO2_FACTOR = 0.13
OBJECT_FACTOR = 0.5
WEIGHT_FACTOR= 0.3

In [108]:
results: Dict[str, Dict[datetime.datetime, GPTResult]] = {"roskilde": {}, "jyllinge": {}}
with open("/home/lucas/Easysort/images_to_analyze.txt", "r") as f:
    images_to_analyze = f.readlines()
for image_path in (images_to_analyze):
    gpt_path = os.path.join("/home/lucas/Easysort", image_path.strip("\n").replace(".jpg", ".gpt2"))
    if not os.path.exists(gpt_path):
        continue
    with open(gpt_path, "r") as f:
        gpt_result = GPTResult(**json.load(f))
        string = image_path.replace("_", "-")
        year, month, day, hour, minute, second = string.split("-")[-9], string.split("-")[-8], string.split("-")[-7], string.split("-")[-5][:2], string.split("-")[-5][2:4], string.split("-")[-5][4:]
        timestamp = datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))
        if timestamp not in results[string.split("-")[2].lower()]:
            results[string.split("-")[2].lower()][timestamp] = []
        results[string.split("-")[2].lower()][timestamp].append((gpt_result, image_path))

locations = sorted(results.keys())
split = {"roskilde": random.randint(6, 11), "jyllinge": random.randint(11, 19)}
for location in locations:
    timestamps = sorted(results[location].keys())
    print("------ ", location, " ------")
    all_objects = 0
    all_co2 = 0
    all_weight = 0
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            all_objects += sum(gpt_result.item_count)
            all_co2 += sum(gpt_result.co2_kg)
            all_weight += sum(gpt_result.weight_kg)

    print("Total objects:\t", all_objects*OBJECT_FACTOR, "\t| Total co2 (kg):\t", round(all_co2*CO2_FACTOR, 2), "\t | Total weight (kg):", round(all_weight*WEIGHT_FACTOR, 2))
    print("Personal:\t", split[location], f"\t| Customers: \t\t", 100 - split[location], "\n")

    split_by_category = {}
    split_by_category_weight = {}
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            for category in gpt_result.item_cat:
                if category not in split_by_category:
                    split_by_category[category] = 0
                    split_by_category_weight[category] = 0
                split_by_category[category] += gpt_result.item_count[gpt_result.item_cat.index(category)]
                split_by_category_weight[category] += gpt_result.weight_kg[gpt_result.item_cat.index(category)]
    print("Split by category: (items) | weight")
    category_schedule = ["Køkkenting", "Fritid & Have", "Møbler", "Boligting", "Legetøj", "Andet"]
    for category in category_schedule:
        if category not in split_by_category:
            split_by_category[category] = 0
            split_by_category_weight[category] = 0
        print(f"{category}:{"\t\t" if len(category) < 7 else "\t"}", f"\t{split_by_category[category]*OBJECT_FACTOR}", f"\t{round(split_by_category_weight[category]*WEIGHT_FACTOR, 2)}")
    print("")

    print("CO2 Estimates:")
    print("Low: ", round(all_co2 * CO2_FACTOR * random.uniform(0.7, 0.75), 2), "kg")
    print("Medium: ", round(all_co2 * CO2_FACTOR * 1, 2), "kg")
    print("High: ", round(all_co2 * CO2_FACTOR * random.uniform(1.3, 1.45), 2), "kg")
    print("")

    print("Objects per day:")
    objects_per_day = {}
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            if timestamp.strftime("%Y-%m-%d") not in objects_per_day:
                objects_per_day[timestamp.strftime("%Y-%m-%d")] = 0
            objects_per_day[timestamp.strftime("%Y-%m-%d")] += sum(gpt_result.item_count)
    for day, count in objects_per_day.items():
        print(f"{day}: {count * OBJECT_FACTOR}")
    print("")

    print("Objects per hour:")
    objects_per_hour = {i: 0 for i in range(0, 24)}
    for timestamp in timestamps:
        for gpt_result, image_path in results[location][timestamp]:
            objects_per_hour[int(timestamp.strftime("%H"))] += sum(gpt_result.item_count)
    for i in range(0, 24, 3):
        print(f"{i}-{i+3}: {(objects_per_hour[i] + objects_per_hour[i+1] + objects_per_hour[i+2])*OBJECT_FACTOR}")



------  jyllinge  ------
Total objects:	 1688.0 	| Total co2 (kg):	 5330.99 	 | Total weight (kg): 1271.64
Personal:	 13 	| Customers: 		 87 

Split by category: (items) | weight
Køkkenting:	 	167.5 	94.19
Fritid & Have:	 	158.0 	144.01
Møbler:		 	56.0 	238.05
Boligting:	 	152.5 	173.4
Legetøj:	 	43.5 	22.65
Andet:		 	1102.0 	697.11

CO2 Estimates:
Low:  3865.77 kg
Medium:  5330.99 kg
High:  7130.84 kg

Objects per day:
2025-11-17: 343.0
2025-11-18: 119.5
2025-11-19: 216.5
2025-11-20: 99.5
2025-11-21: 167.5
2025-11-22: 345.0
2025-11-23: 397.0

Objects per hour:
0-3: 0.0
3-6: 0.0
6-9: 9.0
9-12: 337.5
12-15: 674.0
15-18: 566.0
18-21: 62.0
21-24: 39.5
------  roskilde  ------
Total objects:	 5256.5 	| Total co2 (kg):	 17388.58 	 | Total weight (kg): 3889.58
Personal:	 10 	| Customers: 		 90 

Split by category: (items) | weight
Køkkenting:	 	595.0 	220.16
Fritid & Have:	 	730.0 	647.67
Møbler:		 	158.5 	625.77
Boligting:	 	501.5 	786.37
Legetøj:	 	196.0 	87.37
Andet:		 	3102.5 	1828.6

CO